In [1]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch import FloatTensor
from scipy.stats import skew, kurtosis, entropy
from math import gcd
from functools import reduce
import pywt
from scipy.stats import skew, kurtosis
from scipy.special import entr
from sklearn.feature_selection import mutual_info_regression

# Req for package
sys.path.append("../")
from SkinLearning.NN.Helpers import train, test, DEVICE, set_seed
from SkinLearning.NN.Models import MultiTemporal


torch.backends.cudnn.benchmark = True

In [2]:
set_seed()

In [10]:
# Folder name will correspond to index of sample
class SkinDataset(Dataset):
    def __init__(
        self,
        scaler,
        signalFolder="D:/SamplingResults2",
        sampleFile="../Data/newSamples.pkl",
        extraction_args={},
        runs=range(65535),
        steps=128,    
    ):
        # Load both disp1 and disp2 from each folder
        # Folders ordered according to index of sample
        # Use the corresponding sample as y -> append probe?
        self.input = []
        self.output = []
        
        with open(f"{sampleFile}", "rb") as f:
             samples = pickle.load(f)
        
        self.min = np.min(samples[runs])
        self.max = np.max(samples[runs])
        
        
        for run in tqdm(runs):
            inp = []
            fail = False
            
            files = os.listdir(f"{signalFolder}/{run}/")
            
            if files != ['Disp1.csv', 'Disp2.csv']:
                continue
            
            for file in files:
                a = pd.read_csv(f"{signalFolder}/{run}/{file}")
                a.rename(columns = {'0':'x', '0.1': 'y'}, inplace = True)
                
                # Skip if unconverged
                if a['x'].max() != 7.0:
                    fail = True
                    break

                # Interpolate curve for consistent x values
                xNew = np.linspace(0, 7, num=steps, endpoint=False)
                interped = interp1d(a['x'], a['y'], kind='cubic', fill_value="extrapolate")(xNew)
                    
                
                inp.append(interped.astype("float32"))
            
            if not fail:
                if len(inp) != 2:
                    raise Exception("sdf")

                self.input.append(inp)
                self.output.append(samples[int(run)])
        
        scaler.fit(self.output)
        self.output = scaler.fit_transform(self.output)
        self.output = tensor(self.output).type(FloatTensor)
        
        # Perform WPD on all sets of signals
        # Using the given parameters
        extracted_features = []
        for i, signals in enumerate(self.input):
            extraction_args['signals'] = signals
            extracted_features.append(waveletExtraction(**extraction_args))       
            
        self.input = extracted_features
        del extracted_features
            
        self.input = tensor(np.array(self.input)).type(FloatTensor)
        
        
    def __len__(self):
        return len(self.output)
    
    def __getitem__(self, idx):
        sample = {"input": self.input[idx], "output": self.output[idx]}
        return sample
    
    

In [11]:
"""
    Creates the data set from filtered samples
    Returns the dataset and the scaler
"""
def getDataset(**kwargs):
    # Get filtered data
    if not 'runs' in kwargs.keys():
        with open("../Data/filtered.pkl", "rb") as f:
            runs = pickle.load(f)

        kwargs['runs'] = runs

    scaler = MinMaxScaler()
    dataset = SkinDataset(scaler=scaler, **kwargs)

    return dataset, scaler

In [12]:
"""
    Creates a train/test split from the given data
    Returns train and test data loaders
"""
def getSplit(dataset, p1=0.8):
    train_n = int(p1 * len(dataset))
    test_n = len(dataset) - train_n
    train_set, test_set = random_split(dataset, [train_n, test_n])

    return DataLoader(train_set, batch_size=32, shuffle=True), \
        DataLoader(test_set, batch_size=32, shuffle=True)

In [13]:
def waveletExtraction(
    signals,
    method,
    combined=False,
    wavelet='db4',
    level=3,
    combine_method='concatenate',
    order='freq',
    levels=[3],
    stats=['mean', 'std', 'skew', 'kurtosis'],
    normalization=None,
    flat=False
):
    def get_statistics(coefficients, stats_list):
        features = []
        for stat in stats_list:
            if stat == 'mean':
                features.append(np.mean(coefficients))
            elif stat == 'std':
                features.append(np.std(coefficients))
            elif stat == 'skew':
                features.append(skew(coefficients))
            elif stat == 'kurtosis':
                features.append(kurtosis(coefficients))
        return features

    def extract_features_single_signal(signal, method, wavelet, level, order, levels, stats_list):
        wp = pywt.WaveletPacket(data=signal, wavelet=wavelet, mode='symmetric', maxlevel=level)
        
        if flat:
            return np.array([node.data for node in wp.get_level(level, 'natural')]).flatten()
        
        features = []

        for l in levels:
            coeffs =  wp.get_level(level, order)
            coeffs = np.array([c.data for c in coeffs])

            if method == "energy" or method == "min-max":
                 # Normalise
                coeffs = (coeffs - np.mean(coeffs)) / np.std(coeffs)
                
            if method == 'raw':
                features.extend(np.concatenate(coeffs))
            elif method == 'energy':       
                features.extend([np.sum(np.square(c)) for c in coeffs])
            elif method == 'entropy':
                features.extend([np.sum(entr(np.abs(c))) for c in coeffs])
            elif method == 'min-max':
                features.extend([np.min(c) for c in coeffs] + [np.max(c) for c in coeffs])
            elif method == 'stats':
                for c in coeffs:
                    features.extend(get_statistics(c, stats_list))
        
            # Optional normalisation for raw and energy
            if normalization == "indvidual":
                features = (features - min(features)) / (max(features) - min(features))
                
        return features

    if combined:
        combined_signal = np.concatenate(signals, axis=0)
        combined_features = extract_features_single_signal(combined_signal, method, wavelet, level, order, levels, stats)
        features = combined_features
    else:
        features_list = [extract_features_single_signal(signal, method, wavelet, level, order, levels, stats) for signal in signals]

        if combine_method == 'concatenate':
            features = np.concatenate(features_list)
        elif combine_method == 'interleave':
            features = np.ravel(np.column_stack(features_list))
        else:
            raise ValueError("Invalid combine_method. Choose from 'concatenate' or 'interleave'.")
    
    if normalization == "combined":
        features = (features - np.min(features, axis=0)) / (np.max(features, axis=0) - np.min(features, axis=0))
    
    return features

In [31]:
def create_best(max_level=6):
    def parse_dataset(method, combined, order, normalize, stats):
        key = (method, normalize, stats)
        normalize = key[1]
        stats = key[-1]
        extraction_args = {
            "signals": None,
            "method": method,
            "combined": combined,
            "wavelet": "db4",
            "level": max_level,
            "order": order,
            "levels": [6],
            "normalization": normalize,
            "stats": stats if method == "stats" else None,
        }
        print(stats)
        combined_str = 'combined' if combined else 'separated'
        dataset_dict[(combined_str, order)][key] = getDataset(
            extraction_args=extraction_args,
        )

    dataset_dict = {
        #("combined", "freq"): {},
        ("separated", "natural"): {},
        #("combined", "natural"): {},
       # ("separated", "freq"): {},
    }
    
    parse_dataset(
        "entropy",
        False,
        "natural",
        False,
        ("None")
    )
    
    return dataset_dict

all_datasets = create_best()

None


100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:10<00:00, 218.12it/s]


In [32]:
def create_lengths_and_loaders(datasets, split_ratio=0.8, batch_size=32):
    lengths = {}
    loaders = {}

    for comb in datasets.keys():
        loaders[comb] = {}
        lengths[comb] = {}
        # Get train and test split
        
        for method in datasets[comb].keys():
            train_data, test_data = getSplit(datasets[comb][method][0], split_ratio)

            # Create DataLoaders
            train_loader, test_loader = getSplit(datasets[comb][method][0])

            # Update the loaders dictionary
            loaders[comb][method] = {'train': train_loader, 'test': test_loader}

            # Update the lengths dictionary
            lengths[comb][method] = len(datasets[comb][method][0][0]['input'])

    return lengths, loaders
lengths, loaders = create_lengths_and_loaders(all_datasets)

In [60]:
def train_models(model_type=None, epochs=700):
    comb_losses = {}
    non_comb_losses = {}
    
    for model_type in models.keys() if not model_type else [model_type]:
        non_comb_losses[model_type] = {}
        comb_losses[model_type] = {}

        
        for comb in models[model_type].keys():
            non_comb_losses[model_type][comb] = {}
            comb_losses[model_type][comb] = {}

            for ext_method in models[model_type][comb].keys():
                non_comb_losses[model_type][comb][ext_method] = {}
                comb_losses[model_type][comb][ext_method] = {}

                for fc_type in models[model_type][comb][ext_method].keys():
                    non_comb_losses[model_type][comb][ext_method][fc_type] = {}
                    comb_losses[model_type][comb][ext_method][fc_type] = {}
                    
                    for model_method in models[model_type][comb][ext_method][fc_type].keys():
                        non_comb_losses[model_type][comb][ext_method][fc_type][model_method] = {}
                        comb_losses[model_type][comb][ext_method][fc_type][model_method] = {}
                        
                        for output in models[model_type][comb][ext_method][fc_type][model_method].keys():

                            non_comb_losses[model_type][comb][ext_method][fc_type][model_method][output] = {}
                            comb_losses[model_type][comb][ext_method][fc_type][model_method][output] = {}
                            
                            print(f"Running {model_type} based on {model_method} and {fc_type}, with dataset using {comb} {ext_method}, with {output}")
                            train_loss, val_loss = train(
                                loaders[comb][ext_method]['train'],
                                models[model_type][comb][ext_method][fc_type][model_method][output],
                                val_loader=loaders[comb][ext_method]['test'],
                                LR=0.001,
                                epochs=epochs,
                                early_stopping=True,
                                patience=50
                            )

                            # Stop at 50 epochs
                            # Only add if last lost had potential
                            if comb == "Combined":
                                comb_losses[model_type][comb][ext_method][fc_type][model_method][output] = {
                                    'Train Loss': train_loss,
                                    'Validation Loss': val_loss
                                }
                            else:
                                non_comb_losses[model_type][comb][ext_method][fc_type][model_method][output] = {
                                    'Train Loss': train_loss,
                                    'Validation Loss': val_loss
                                }


In [63]:
def test_models(model_type=None):
    temp_dict = {}
    for model_type in models.keys() if not model_type else [model_type]:
        for comb in models[model_type].keys():

            for ext_method in models[model_type][comb].keys():
                for fc_type in models[model_type][comb][ext_method].keys():

                    for model_method in models[model_type][comb][ext_method][fc_type].keys():
                        for output in models[model_type][comb][ext_method][fc_type][model_method].keys():
                            #print(f"Running {model_type} based on {model_method}, with dataset using {comb} {ext_method} using {output}")

                            validation = test(
                                    loaders[comb][ext_method]['test'],
                                    models[model_type][comb][ext_method][fc_type][model_method][output],
                                    all_datasets[comb][ext_method][1]
                            )
                            
                            """                            losses = comb_losses if comb == "combined" else non_comb_losses

                            print(
                                "Final training loss: ",
                                losses[model_type][comb][ext_method][fc_type][model_method]['Train Loss'][-1]
                            )"""
                            """print(
                                f"{fc_type}, MAPE: {validation[0]}, MAE: {validation[2]}\n"
                            )"""
                            
                            
                            try:
                                temp_dict[(model_type, comb, model_method, ext_method, output)][fc_type] = f"{fc_type}, MAPE: {validation[0]}, MAE: {validation[2]}"
                            except:
                                temp_dict[(model_type, comb, model_method, ext_method, output)] = {}
                                temp_dict[(model_type, comb, model_method, ext_method, output)][fc_type] = f"{fc_type}, MAPE: {validation[0]}, MAE: {validation[2]}"
    for key in temp_dict.keys():
        print(f"{key[0]} based on {key[2]}, with dataset using {key[1]} {key[3]} using {key[4]}")
        for strs in temp_dict[key].values():
              print(strs)
        print("\n")

In [93]:
class MultiTemporal(nn.Module):
    def __init__(
        self,
        conv=True,
        input_size=7,
        hidden_size=256,
        single_fc=True,
        out="f_hidden",
        layers=1,
        temporal_type="RNN",
        fusion_method="concatenate"
    ):
        super(MultiTemporal, self).__init__()

        self.hidden_size = hidden_size
        self.out = out
        self.temporal_type = temporal_type
        self.fusion_method = fusion_method
        self.input_size = input_size
        self.conv = conv
        
        if conv:
            self.cnn = deepcopy(best_CNN)
        

        if temporal_type == "RNN":
            net = nn.RNN
        elif temporal_type == "LSTM":
            net = nn.LSTM
        elif temporal_type == "GRU":
            net = nn.GRU
        else:
            raise Exception("Not a valid NN type.")

        if fusion_method == 'concatenate':
            self.net = net(input_size, hidden_size, layers, batch_first=True)
        elif fusion_method == 'multi_channel':
            self.net = net(2, hidden_size, layers, batch_first=True)
        elif fusion_method == 'independent':
            self.net = net(input_size, hidden_size, layers, batch_first=True)
        else:
            raise ValueError("Invalid method. Choose from 'concatenate', 'multi_channel', or 'independent'.")
        
        # Check size of output to determine FC input
        input_tensor = torch.zeros(
            32,
            512,
            input_size if fusion_method != "independent" else input_size
        )
        
        if self.temporal_type == "LSTM":
            output, (hidden, _) = self.net(input_tensor)
        else:
            output, hidden = self.net(input_tensor)
            
        fc_in = hidden_size

        if fusion_method == "independent":
            fc_in *= 2
            
        if out == "h+o":
            fc_in *= 2
            
        print("fc in", fc_in)
            
        if single_fc:
            self.fc = nn.Linear(fc_in*layers, 6)
        else:
            self.fc = nn.Sequential(
                nn.Linear(256, 128),
                nn.ReLU(),
                nn.Linear(128 , 64),
                nn.ReLU(),
                nn.Linear(64, 6),   
            )
            
            if fc_in > hidden_size:
                if fc_in == 4096:
                    print("Starts 4096")
                    init_layers = nn.Sequential(
                        nn.Linear(4096, 2048),
                        nn.ReLU(),
                        nn.Linear(2048, 1024),
                        nn.ReLU(),
                        nn.Linear(1024 , 512),
                        nn.ReLU(),
                        nn.Linear(512, 256), 
                    )
                elif fc_in == 512:
                    print("Starts 512")
                    init_layers = nn.Sequential(
                        nn.Linear(512, 256),
                        nn.ReLU()
                    )
                elif fc_in == 2048:
                    print("Starts 2048")
                    init_layers = nn.Sequential(
                        nn.Linear(2048, 1024),
                        nn.ReLU(),
                        nn.Linear(1024, 512),
                        nn.ReLU(),
                        nn.Linear(512, 256),
                        nn.ReLU()
                    )
                
                self.fc = nn.Sequential(init_layers, self.fc)
            print("\n")   

    def forward(self, x):
        batch_size = x.shape[0]
        
        x = x.reshape(batch_size, 2, -1)
        if self.conv:
            signal1 = self.cnn(x)
            signal2 = self.cnn(x)
        else:
            x = x.reshape(batch_size, -1, self.input_size)

        def getOutputs(inp):
            if self.temporal_type == "LSTM":
                o, (h, _) = self.net(inp)
            else:
                o, h = self.net(inp)
            return o, h

        
        if self.fusion_method == 'multi_channel':
            o, h = getOutputs(x.view(batch_size, -1, 2))
        elif self.fusion_method == 'independent':
            signal_size = self.input_size//2
            #signal1 = x[..., :signal_size].reshape(batch_size, -1, signal_size)
            #signal2 = x[..., signal_size:].reshape(batch_size, -1, signal_size)
            
            o1, h1, = getOutputs(signal1)
            o2, h2 = getOutputs(signal2)
        else:
            o, h = getOutputs(x)
        
        
        if self.out == "f_hidden":
            if self.fusion_method == "independent":
                x = torch.concat(
                    [h1[-1], h2[-1]], dim=1
                    ).reshape(batch_size, -1)
            else:
                x = h[-1].reshape(batch_size, -1)
        elif self.out == "hidden":
            if self.fusion_method == "independent":
                x = torch.concat(
                    [h1, h2], dim=1
                    ).reshape(batch_size, -1)
            else:   
                x = h.reshape(batch_size, -1)
        elif self.out == "f_output":
            if self.fusion_method == "independent":
                x = torch.concat(
                    [o1[:, -1, :], o2[:, -1, :]], dim=1
                    ).reshape(batch_size, -1)
            else:
                x = o[:, -1, :].reshape(batch_size, -1)
        elif self.out == "output":
            if self.fusion_method == "independent":
                x = torch.concat(
                    [o1, o2], dim=1
                    ).reshape(batch_size, -1)
            else:
                x = o.reshape(batch_size, -1)
        elif self.out == "h+o":
                if self.fusion_method == "independent":
                    x1 = torch.concat(
                        [h1[-1], o1[:, -1, :]], dim=1
                        )
                    
                    x2 = torch.concat(
                        [h2[-1], o2[:, -1, :]], dim=1
                        )
                    
                    x = torch.concat([x1, x2], dim=1
                        ).view(o2.size(0), -1)
                else:
                    x = torch.concat([h[-1], o[:, -1, :]], dim=1).view(o.size(0), -1)
            
        x = self.fc(x)
        return x

In [94]:
from copy import deepcopy

best_CNN = nn.Sequential(
    nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=5, stride=2),

    nn.Conv1d(128, 64, kernel_size=3, padding=1, bias=False),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2, stride=2),

    nn.Conv1d(64, 32, kernel_size=3, padding=1, bias=False),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2, stride=2),
)
models = {}

# Get an LSTM and RNN of each type, paired with each type of dataset
for model_type in ['LSTM']:#, 'GRU', 'RNN']:
    models[model_type] = {}
    
    def get_models():
        models[model_type][comb][ext_method][fc_type][model_method] = {}
        for out in ['f_hidden']:
            #for model_method in ['concatenate', 'independent']:#, 'multi_channel', 'independent']:
            #print(fc_type)
            models[model_type][comb][ext_method][fc_type][model_method][out]= \
            MultiTemporal(
                fusion_method=model_method,
                temporal_type=model_type,
                single_fc=True if fc_type == 'Single FC' else False,
                out=out
            )
            print(lengths[comb][ext_method], 2*lengths[comb][ext_method])

    for comb in all_datasets.keys():
        models[model_type][comb] = {}
        
        for ext_method in all_datasets[comb].keys():
            models[model_type][comb][ext_method] = {}
            for fc_type in ['Multi FC']:          
                models[model_type][comb][ext_method][fc_type] = {}     
                
                if ext_method == ('raw', False, 'None'):
                    for model_method in ['independent']:
                        get_models()
                else:
                    model_method = "independent"
                    get_models()

fc in 512
Starts 512


128 256


In [95]:
train_models()

Running LSTM based on independent and Multi FC, with dataset using ('separated', 'natural') ('entropy', False, 'None'), with f_hidden
Using: cuda


100%|█████████████████████████| 56/56 [00:00<00:00, 79.37batch/s, counter=5, epoch=67, lastLoss=0.0594, valLoss=0.0659]


100%|███████████████████████| 56/56 [00:00<00:00, 87.23batch/s, counter=13, epoch=135, lastLoss=0.0482, valLoss=0.0585]


100%|████████████████████████| 56/56 [00:00<00:00, 81.63batch/s, counter=2, epoch=203, lastLoss=0.0407, valLoss=0.0589]


100%|███████████████████████| 56/56 [00:00<00:00, 84.09batch/s, counter=13, epoch=271, lastLoss=0.0362, valLoss=0.0483]


100%|███████████████████████| 56/56 [00:00<00:00, 81.03batch/s, counter=49, epoch=322, lastLoss=0.0297, valLoss=0.0482]


Early stopping after 323 epochs
Average train loss: 0.023485059879758227
Average validation loss: 0.02825133689705814


In [96]:
test_models()

LSTM based on independent, with dataset using ('separated', 'natural') ('entropy', False, 'None') using f_hidden
Multi FC, MAPE: 91.56067562103271, MAE: 0.04947319179773331


